# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 4 2022 10:12AM,243444,19,8360377,"Graystone, LLC",Released
1,Aug 4 2022 9:58AM,243443,10,8360354,MedStone Pharma LLC,Released
2,Aug 4 2022 9:58AM,243443,10,8360355,MedStone Pharma LLC,Released
3,Aug 4 2022 9:58AM,243443,10,8360356,MedStone Pharma LLC,Released
4,Aug 4 2022 9:58AM,243443,10,8360357,MedStone Pharma LLC,Released
5,Aug 4 2022 9:58AM,243443,10,8360358,MedStone Pharma LLC,Released
6,Aug 4 2022 9:58AM,243443,10,8360359,MedStone Pharma LLC,Released
7,Aug 4 2022 9:56AM,243442,10,BPS20002,Beach Patient Assistance,Released
8,Aug 4 2022 9:45AM,243441,10,Stone8360369,Acute Outpatient Solutions,Released
9,Aug 4 2022 9:41AM,243440,12,8360368,LF of America Corp.,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,243440,Executing,1
23,243441,Released,1
24,243442,Released,1
25,243443,Released,6
26,243444,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243440,1.0,NaN
243441,NaN,1.0
243442,NaN,1.0
243443,NaN,6.0
243444,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243355,2.0,1.0
243396,0.0,5.0
243403,12.0,1.0
243414,0.0,1.0
243415,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243355,2,1
243396,0,5
243403,12,1
243414,0,1
243415,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243355,2,1
1,243396,0,5
2,243403,12,1
3,243414,0,1
4,243415,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243355,2,1
1,243396,,5
2,243403,12,1
3,243414,,1
4,243415,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 4 2022 10:12AM,243444,19,"Graystone, LLC"
1,Aug 4 2022 9:58AM,243443,10,MedStone Pharma LLC
7,Aug 4 2022 9:56AM,243442,10,Beach Patient Assistance
8,Aug 4 2022 9:45AM,243441,10,Acute Outpatient Solutions
9,Aug 4 2022 9:41AM,243440,12,LF of America Corp.
10,Aug 4 2022 9:39AM,243439,10,Emerginnova
11,Aug 4 2022 9:36AM,243438,10,ISDIN Corporation
78,Aug 4 2022 9:26AM,243436,10,"Nextsource Biotechnology, LLC"
79,Aug 4 2022 9:25AM,243435,10,Bio-PRF
94,Aug 4 2022 9:24AM,243434,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 4 2022 10:12AM,243444,19,"Graystone, LLC",,1
1,Aug 4 2022 9:58AM,243443,10,MedStone Pharma LLC,,6
2,Aug 4 2022 9:56AM,243442,10,Beach Patient Assistance,,1
3,Aug 4 2022 9:45AM,243441,10,Acute Outpatient Solutions,,1
4,Aug 4 2022 9:41AM,243440,12,LF of America Corp.,1,
5,Aug 4 2022 9:39AM,243439,10,Emerginnova,,1
6,Aug 4 2022 9:36AM,243438,10,ISDIN Corporation,,67
7,Aug 4 2022 9:26AM,243436,10,"Nextsource Biotechnology, LLC",,1
8,Aug 4 2022 9:25AM,243435,10,Bio-PRF,,15
9,Aug 4 2022 9:24AM,243434,12,Hush Hush,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 10:12AM,243444,19,"Graystone, LLC",1,
1,Aug 4 2022 9:58AM,243443,10,MedStone Pharma LLC,6,
2,Aug 4 2022 9:56AM,243442,10,Beach Patient Assistance,1,
3,Aug 4 2022 9:45AM,243441,10,Acute Outpatient Solutions,1,
4,Aug 4 2022 9:41AM,243440,12,LF of America Corp.,,1
5,Aug 4 2022 9:39AM,243439,10,Emerginnova,1,
6,Aug 4 2022 9:36AM,243438,10,ISDIN Corporation,67,
7,Aug 4 2022 9:26AM,243436,10,"Nextsource Biotechnology, LLC",1,
8,Aug 4 2022 9:25AM,243435,10,Bio-PRF,15,
9,Aug 4 2022 9:24AM,243434,12,Hush Hush,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 10:12AM,243444,19,"Graystone, LLC",1,
1,Aug 4 2022 9:58AM,243443,10,MedStone Pharma LLC,6,
2,Aug 4 2022 9:56AM,243442,10,Beach Patient Assistance,1,
3,Aug 4 2022 9:45AM,243441,10,Acute Outpatient Solutions,1,
4,Aug 4 2022 9:41AM,243440,12,LF of America Corp.,,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 10:12AM,243444,19,"Graystone, LLC",1.0,NaN
1,Aug 4 2022 9:58AM,243443,10,MedStone Pharma LLC,6.0,NaN
2,Aug 4 2022 9:56AM,243442,10,Beach Patient Assistance,1.0,NaN
3,Aug 4 2022 9:45AM,243441,10,Acute Outpatient Solutions,1.0,NaN
4,Aug 4 2022 9:41AM,243440,12,LF of America Corp.,NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 10:12AM,243444,19,"Graystone, LLC",1.0,0.0
1,Aug 4 2022 9:58AM,243443,10,MedStone Pharma LLC,6.0,0.0
2,Aug 4 2022 9:56AM,243442,10,Beach Patient Assistance,1.0,0.0
3,Aug 4 2022 9:45AM,243441,10,Acute Outpatient Solutions,1.0,0.0
4,Aug 4 2022 9:41AM,243440,12,LF of America Corp.,0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2434307,105.0,6.0
12,730307,6.0,12.0
16,243428,2.0,0.0
19,973632,33.0,28.0
20,730252,2.0,13.0
21,243414,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2434307,105.0,6.0
1,12,730307,6.0,12.0
2,16,243428,2.0,0.0
3,19,973632,33.0,28.0
4,20,730252,2.0,13.0
5,21,243414,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,105.0,6.0
1,12,6.0,12.0
2,16,2.0,0.0
3,19,33.0,28.0
4,20,2.0,13.0
5,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,105.0
1,12,Released,6.0
2,16,Released,2.0
3,19,Released,33.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Executing,6.0,12.0,0.0,28.0,13.0,0.0
Released,105.0,6.0,2.0,33.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Executing,6.0,12.0,0.0,28.0,13.0,0.0
1,Released,105.0,6.0,2.0,33.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Executing,6.0,12.0,0.0,28.0,13.0,0.0
1,Released,105.0,6.0,2.0,33.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()